<a href="https://colab.research.google.com/github/alidarvishh2492/HW_05/blob/main/HW_05_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from math import sqrt, pi


In [2]:
TRAIN_FILE = "iris_dataset.csv"
TEST_FILE  = "iris_test_samples.csv"

In [3]:
df = pd.read_csv(TRAIN_FILE)
df_test = pd.read_csv(TEST_FILE)

In [4]:
feature_column = list(df.columns[:4])
label_columns = df.columns[-1]

In [5]:
groups = df.groupby(label_columns)
results = {}
for cls, g in groups:
    results[cls] = {}
    for f in feature_column:
        values = g[f].values
        results[cls][f] = {
            'mean': float(np.mean(values)),
            'var':   float(np.var(values)),
            'min':   float(np.min(values)),
            'max':   float(np.max(values)),
            'range': float(np.max(values) - np.min(values))
        }

In [6]:
rows = []
for cls in results:
    for f in results[cls]:
        s = results[cls][f]
        rows.append({'class': cls, 'feature': f, **s})
results_df = pd.DataFrame(rows)
results_df

,class,feature,mean,var,min,max,range
0,0,sepal length (cm),5.006,0.121764,4.3,5.8,1.5
1,0,sepal width (cm),3.428,0.140816,2.3,4.4,2.1
2,0,petal length (cm),1.462,0.029556,1.0,1.9,0.9
3,0,petal width (cm),0.246,0.010884,0.1,0.6,0.5
4,1,sepal length (cm),5.936,0.261104,4.9,7.0,2.1
5,1,sepal width (cm),2.770,0.096500,2.0,3.4,1.4
6,1,petal length (cm),4.260,0.216400,3.0,5.1,2.1
7,1,petal width (cm),1.326,0.038324,1.0,1.8,0.8
8,2,sepal length (cm),6.588,0.396256,4.9,7.9,3.0
9,2,sepal width (cm),2.974,0.101924,2.2,3.8,1.6


In [7]:
feature = feature_column[2]
classes = sorted(results.keys())

predicted = []

for x in df[feature]:
    probs = {}

    for cls in classes:
        mu = results[cls][feature]['mean']
        var = results[cls][feature]['var']

        coefficient = 1 / np.sqrt(2 * np.pi * var)
        exponent = -0.5 * ((x - mu) ** 2) / var
        p = coefficient * np.exp(exponent)

        probs[cls] = p

    best_class = max(probs, key=probs.get)
    predicted.append(best_class)

df['predicted_feature'] = predicted
accuracy = (df['predicted_feature'] == df[label_columns]).mean() * 100
accuracy

np.float64(95.33333333333334)

In [8]:
predicted_test = []

for x in df_test['petal_length']:
    p_per_class = {}

    for cls in classes:
        mu = results[cls][feature]['mean']
        var = max(results[cls][feature]['var'], 1e-9)

        coefficient = 1.0 / sqrt(2 * pi * var)
        exponent = -0.5 * ((x - mu) ** 2) / var
        p = coefficient * np.exp(exponent)

        p_per_class[cls] = p

    chosen_class = max(p_per_class, key=p_per_class.get)
    predicted_test.append(chosen_class)

df_test['predicted_feature'] = predicted_test

if 'label' in df_test.columns:
    correct_test = (df_test['predicted_feature'] == df_test['label']).sum()
    predicted_test_feature = correct_test / len(df_test) * 100
else:
    predicted_test_feature = None

predicted_test_feature

np.float64(100.0)

In [9]:
df.to_csv("iris_train_with_preds.csv", index=False)
df_test.to_csv("iris_test_with_preds.csv", index=False)
print("Saved annotated train/test CSVs.")


Saved annotated train/test CSVs.
